# Pokémon Card Detection Pipeline
This notebook guides you through detecting, cropping, and hashing Pokémon cards in a binder photo.

## 1. Collect Training Data
- Collect images of Pokémon cards from binder photos (9 cards typically).
- Label them using [LabelImg](https://github.com/tzutalin/labelImg) or [Roboflow](https://roboflow.com/).
- Annotations should include bounding boxes around each card (labeled as `card`).

## 2. Set Up the YOLOv8 Model
Install YOLOv8 and create the dataset folder structure.

```bash
pip install ultralytics
```

### Directory structure:
```
datasets/cards/images/train/
datasets/cards/labels/train/
datasets/cards/images/val/
datasets/cards/labels/val/
```

### YAML configuration (`data.yaml`):
```yaml
path: datasets/cards
train: images/train
val: images/val
names:
  0: card
```

### Training the Model:
```python
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model.train(data='datasets/cards/data.yaml', epochs=100)
```

## 3. Crop Detected Cards
Once the model detects the cards, we extract the bounding boxes and crop each detected card.
```python
import cv2
# img: your original image
# boxes: detections
for box in boxes:
    x1, y1, x2, y2 = box.xyxy[0]
    card_crop = img[int(y1):int(y2), int(x1):int(x2)]
# Save or hash the card_crop
```

## 4. Hash and Compare Cropped Cards
Use perceptual hashing to generate unique hashes for each card, then compare them against a database of card hashes.
```python
from PIL import Image
import imagehash
hash = imagehash.phash(Image.fromarray(card_crop))

# Compare with database hashes
matches = []
for db_hash in database_hashes:
    if hash - db_hash < 5:  # Hamming distance threshold
        matches.append(db_hash)
```
You can tweak the distance threshold to match cards with high accuracy.

## 5. Save and Compare with Database
To keep track of hashes for matching, save each card's hash into a database or simple list.
You can use this for future comparisons and quick lookups when detecting cards in new binder images.